In [1]:
import sys
import numpy as np
import time
import pickle
import pandas as pd
import copy
import matplotlib.pyplot as plt

import allensdk
import pandas as pd

In [2]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

# Mouse Connectivity

This notebook analyzes the mouse connectivity data for the purpose of calculating the information flow in the brain. We reply on Allen brain Atlas API  (allensdk package) with some custom codes.

As a brief summary, Allen brain Atlas: Mouse Brain Connectivity contains data on  experiments on the mouse brain. Each experiment has an associated (unique) injection site where green fluroescent protein virus is injected. This injection infects the cell-bodies of the neurons in the injection site, and they propagates along the axons (carrier of the action potential of neurons) where they reach its terminal and emit green lights. These signals then are imaged through green-flurescent imaging and mapped accordingly. More information can be found on0 http://connectivity.brain-map.org/. Some defintions relevant to the throughout the notbook are defined as follows.

projection density = sum of detected pixels / sum of all pixels in division  
projection energy = sum of detected pixel intensity / sum of all pixels in division  
injection_fraction = fraction of pixels belonging to manually annotated injection site  
injection_density = density of detected pixels within the manually annotated injection site  
injection_energy = energy of detected pixels within the manually annotated injection site  
data_mask = binary mask indicating if a voxel contains valid data (0=invalid, 1=valid). Only valid voxels should be used for analysis  


This notebook carefully combines multiple experiments to create a connectome data. This data is then traslated into the information flow data, namely we determine how much information flows at a regional level.

## Fetch all the Experiment list


First we create MouseConnectivityCache instance.

Fome this we may fetch the experiment list. One in dataFrame and one in dictionary format.

In [3]:
# MouseConnectivityCache allows access to data stored within
mcc=MouseConnectivityCache(manifest_file='connectivity/mouse_connectivity_manifest.json')

#get the id of all experiments.
#Create one for Pandas dataframe and the other in terms of dictionary
experiment_list = mcc.get_experiments(dataframe=True)
experiment_list_dict=mcc.get_experiments(dataframe=False)

Sample of some of the entries of the experiment_list looks like:

In [4]:
experiment_list[:3]

,gender,id,injection_structures,injection_volume,injection_x,injection_y,injection_z,primary_injection_structure,product_id,specimen_name,strain,structure_abbrev,structure_id,structure_name,transgenic_line,transgenic_line_id
id,,,,,,,,,,,,,,,,
527712447,F,527712447,"[502, 926, 1084, 484682470]",0.006655,9240,3070,8990,502,5,Penk-IRES2-Cre-neo-249961,C57BL/6J,SUB,502,Subiculum,Penk-IRES2-Cre-neo,298725927.0
301875966,M,301875966,"[574, 931]",0.105746,9170,6850,6200,574,5,Gabrr3-Cre_KC112-3467,C57BL/6J,PG,931,Pontine gray,Gabrr3-Cre_KC112,177838877.0
520336173,M,520336173,"[1, 210, 491, 525, 1004]",0.025762,7810,6550,6450,1,5,Hdc-Cre_IM1-204103,None,TMv,1,"Tuberomammillary nucleus, ventral part",Hdc-Cre_IM1,177839494.0


where the columns are:

Each experiment entry contains the following columns:

In [466]:
for i in experiment_list_dict[0].keys():
    print(i)

injection_y
injection_x
injection_z
product_id
structure_name
gender
transgenic_line
injection_volume
structure_abbrev
transgenic_line_id
primary_injection_structure
strain
id
specimen_name
injection_structures
structure_id


with the number of separate experiments being:

In [491]:
len(experiment_list_dict)

2995

This is for allensdk 0.14.5. Number of experiments may vary depending on the version of the package. Note that each injection sites has a unique experiemnt, that is, no more than one injection sites were allowed in a single mouse brain.

## Annotation Volume

We then download annotation volume for future uses. This comes in the form of an array where structures are denoted by the values of the elements. Spatial coordinates on the other hand, are represented by the indices of the array. First create an instance using the MouseConnectivityCache (mcc)


In [8]:
annt=mcc.get_annotation_volume()[0]

the annotation volume has the shape

In [9]:
annt.shape

(528L, 320L, 456L)

for future, we collect the id of all the structures present in the annt volume

In [10]:
annt_structures=[]
for i in range(annt.shape[0]):
    for j in range(annt.shape[1]):
        for k in range(annt.shape[2]):
            if annt[i,j,k] not in annt_structures:
                annt_structures.append(annt[i,j,k])


## Structure_tree

Allenmouse structure ontology is organized as a tree where each edge indicates the physical containment. Structuretree can be used to fetch properties of a given structure, including its id, name, acronym, etc..

In [11]:
structure_tree=mcc.get_structure_tree()

sample of each structure is shown:

In [12]:
pd.DataFrame(structure_tree.get_structures_by_name(['Thalamus','Hypothalamus']))

,acronym,graph_id,graph_order,id,name,rgb_triplet,structure_id_path,structure_set_ids
0,TH,1,641,549,Thalamus,"[255, 112, 128]","[997, 8, 343, 1129, 549]","[2, 112905828, 691663206, 12, 184527634, 11290..."
1,HY,1,715,1097,Hypothalamus,"[230, 68, 56]","[997, 8, 343, 1129, 1097]","[2, 112905828, 691663206, 12, 184527634, 11290..."


where the columns are:

In [13]:
for i in structure_tree.get_structures_by_name(['Thalamus'])[0]:
    print(i)

rgb_triplet
graph_id
name
acronym
graph_order
structure_id_path
structure_set_ids
id


## structure set

On top of strucure tree, AllenmouseSDK provides a convenient way to group structures together through the structure sets.
For example, if i wish to fetch structures trees beonging to Midbrain, I can easily do so.

In [14]:
from allensdk.api.queries.ontologies_api import OntologiesApi

#initialize oapi_inst instance
oapi_inst=OntologiesApi()

In [15]:
structure_set_ids=structure_tree.get_structure_sets()
structure_set=pd.DataFrame(oapi_inst.get_structure_sets(structure_set_ids))
structure_set

,description,id,name
0,List of structures in Isocortex layer 5,667481446,Isocortex layer 5
1,List of structures in Isocortex layer 6b,667481450,Isocortex layer 6b
2,Summary structures of the cerebellum,688152368,Cerebellum
3,List of structures for ABA Differential Search,12,ABA - Differential Search
4,List of valid structures for projection target...,184527634,Mouse Connectivity - Target Search
5,Structures whose surfaces are represented by a...,691663206,Mouse Brain - Has Surface Mesh
6,Summary structures of the midbrain,688152365,Midbrain
7,Summary structures of the medulla,688152367,Medulla
8,Summary structures of the striatum,688152361,Striatum
9,Structures representing subdivisions of the mo...,687527945,Mouse Connectivity - Summary


As shown above, we can select the structure set based on its description and fetch the desired collection of structures>

Here our interest is in the projection research, therefore we fetch the summary structures from 184527634 (Mouse Connectivity - Target Search)

In [16]:
projection_structures_target=pd.DataFrame(structure_tree.get_structures_by_set_id([184527634]))

### how many structures in each?

Out of curiosity, we explore how many structures are contained in each structure set:

In [19]:
#very clever name for a variable indicating exactly what it is used for.
printme={'experiment id':[],'number of structures':[]}

for i in structure_set[structure_set.columns[1]]:  #column 1 is id
    printme['experiment id'].append(i)
    printme['number of structures'].append(len(structure_tree.get_structures_by_set_id([i])))
print pd.DataFrame(printme)

    experiment id  number of structures
0       667481446                    43
1       667481450                    43
2       688152368                    18
3              12                   843
4       184527634                   840
5       691663206                   840
6       688152365                    39
7       688152367                    45
8       688152361                    14
9       687527945                   293
10      688152359                    15
11      514166994                     7
12      688152358                    11
13      167587189                   316
14      667481445                    26
15      687527670                    12
16      688152362                     9
17      114512892                    79
18      112905813                   111
19             10                    74
20      112905828                   397
21      667481449                    43
22              3                    51
23      667481440                    43


# Maps

Each structure can be denoted by all kinds of variables such as its id, acronym, or its full name. Therefore it is convenient to define maps that will take care of this. We want the map to function as a converter between variables.


In [20]:
projection_structures_inj=pd.DataFrame(projection_structures_inj)

acronym=projection_structures_inj['acronym'].values.tolist()
acronym_to_id={}
acronym_to_name={}
acronym_to_volume={}
id_to_idpath={}
id_to_name={}

for i in range(len(acronym)):
    a=projection_structures_inj['acronym'][i] #structure acronym
    b=projection_structures_inj['structure_id_path'][i][-1] #sturcture id
    c=projection_structures_inj['name'][i] #structure name
    d=projection_structures_inj['structure_id_path'][i] #structure id path
    acronym_to_id[a]=b
    acronym_to_name[a]=c
    id_to_name[b]=c
    id_to_idpath[b]=d


### structure volumes

as far as I am aware, getting the structure volume is not straight forward

define inverse map for the dictionary

In [21]:
def invert_me(arg):
    """inverts a map
        args:
            arg(dictionary): input dictionary
        returns:
            inverted input dictionary"""
    return {v: k for k,v in arg.iteritems()}

In [22]:
id_to_acronym=invert_me(acronym_to_id)
name_to_acronym=invert_me(acronym_to_name)
name_to_id=invert_me(id_to_name)

Since the structures organized as trees, it would be nice to have a dictionary that can return all its daughter structures (or all structures beneath it). Hence here we define a dictionary that contains its direct substructures (only one level below): Substr

In [23]:
projection_structures_inj.columns[-2]

'structure_id_path'

In [24]:
Substr={}
Substr[997]=[]
for acr in acronym:
    Substr[acronym_to_id[acr]]=[]

    
unknown_structures=[]


for i in projection_structures_inj['structure_id_path']:
    try:
        Substr[i[-2]].append(i[-1])
    except:
        unknown_structures.append(i[-2])

In [25]:
unknown_structures

[81, 81]

Unknown structure with id 81

Now we define functions that unpacks a list of all structures below a particular structure using Substr

In [26]:
def Unpack(structure_id,MST,include_self=False):
    """returns a list of substructures of a given structure
        args:
            structures_id(int): id of the structure
            MST(dictionary): My Structure Tree
            include_self(bool): decide whether to include the initial structure, default=False
        
        returns:
            king(list) """
    #Check for Error
    #if int(structure_id)!=structure_id:
    #    raise ValueError("not a valid structure_id. structure_id must be an integer")
    
    
    unpacked=[]
    
    
    
    def Unpack_(structure_id,MST):
        """Primarily used for 'recursion' of the unpack"""
        
        #check error
        #if structure_id  in structure_id_list_exception:
        #    raise ValueError("structure id not recognized")
         
        leng=len(MST[structure_id])

        if leng>=1:
            unpacked.append(structure_id)
            for i in MST[structure_id]:
                Unpack_(i,MST)

        elif leng==0:
            unpacked.append(structure_id)
        

    Unpack_(structure_id,MST)
    
    if include_self:
        pass
    elif not include_self:
        unpacked.remove(structure_id)
    
    return unpacked

In [27]:
name_to_id['Thalamus']

549

In [28]:
len(Unpack(997,Substr,include_self=False))

838

## Injection Fraction 

We wish to find how many fractions of the total injection volume of an experiment belong to the structure of interest.

id_to_idpath defined earlier is not complete without structure id 997 and 0

In [29]:
id_to_idpath[0]=[]
id_to_idpath[997]=[997]

structure_id_list_exception=[81]

In [30]:
def create_structure_dictionary(annt_values,inj_density_values,include_density=True):
    """args:
        annt_values: (array, int 32): array of structure ids obtained from annotation volume[injection_mask]
        inj_density_values (array): 
        
        returns:
            structuer_dict(dictionary): a map from a structure id to its fraction in the injection volume
            """
    
    #check that annt_values and inj_density are the same size
    if len(annt_values)!=len(inj_density_values):
        raise ValueError('annt volume and inj density must be the same size')
    
    structure_dict={}
    leng=len(annt_values)
    exception_count=0
    exception_id_list=[]
    
    if include_density:
        
        for i in range(leng):
            
            if annt_values[i] in structure_id_list_exception:
                exception_id_list.append(annt)
                pass
            elif annt_values[i] not in structure_id_list_exception:
                if annt_values[i] not in structure_dict:
                    structure_dict[annt_values[i]]=0
                elif annt_values[i] in structure_dict:
                    structure_dict[annt_values[i]]+=inj_density_values[i]
                    
                    
                    
                    
                    
        #need to add the id of each superstructures of structure_dict
        extra=[]
        
        for i in structure_dict:
            try:
                sup=id_to_id_path[i]
                for j in sup:
                    if j not in structure_dict:
                        extra.append(j)
            except:
                exception_count+=1
                pass
        for i in extra:
            structure_dict[i]=0
                
        return structure_dict,exception_count,exception_id_list
    
    
    elif not include_density:
        for i in arg:
            try:
                if i not in structure_dict:
                    structure_dict[i]=0
            except:
                exception_count+=1
                pass

            if i in structure_dict:
                structure_dict[i]+=1
        return structure_dict,exception_count,exception_id_list
    
def generate_experiment_structure_fractions( experiment_list, annt,MST,include_density=True):
    """returns a dictionary that contains fraction information about the injection volume of the structure"""
    
    """args:
        experiment_list(list): list of experiment ids
        annt (ndarray) annotation volume
        include_density (bool): decide whether to calculate using the density of simple volume number.
        Default=True"""
    
    #returning dictionary
    return_dict={}
    
    #iterate over the given experiment list.
    for i in experiment_list:
        
        #fetch injection density information
        inj_density=mcc.get_injection_density(i)[0]
        #create booleean array that is True on the injection sites
        inj_density_bool=inj_density!=0.0
        
        #np.where returns a tuple (x,y,z) for 3d, which are to be unpacked immediately
        a,b,c=np.where(inj_density_bool)
        #using the voxel coordinates a,b,c, find density values at each coordinate. 
        inj_density_values=inj_density[a,b,c]
        #and annotation values that contain id list
        annt_values=annt[a,b,c]
        
        
        
        #call the function create_structure_dictionary that analyzies the fraction information
        sub_dict,exception_count_sub,exception_id_list_sub=create_structure_dictionary(annt_values,inj_density_values,include_density=True)
        
        
        
        for p in sub_dict:
            if not include_density:
                leng=annt_values.shape[0]
                sub_dict[p]=float(sub_dict[p])/leng
            elif include_density:
                #normalize the fraction
                sub_dict[p]=float(sub_dict[p])/sum(inj_density_values)
        
        return_dict[i]=sub_dict
    
    exception_count=0
    exception_id_list=[]
    
    # now add the fraction of substructures to the superstructure
    #normal copy would not really 'copy' the dictionary. Thus use copy.deepcopy
    return_dict_modified=copy.deepcopy(return_dict)
    for i in return_dict:
        for j in return_dict[i]:
            #call unpack function to list all the substructures
            if j in projection_structures_inj:
                unpackedpre=Unpack(j,MST,include_self=True)

                for k in unpackedpre:
                    if j!=k:
                        try:
                            aa=return_dict[i][k].copy()
                            return_dict_modified[i][j]+=aa
                        except:
                            exception_count+=1
                            pass
            else:
                exception_count+=1
                exception_id_list.append(j)
                pass
    return return_dict_modified,exception_count,exception_id_list,exception_count_sub,exception_id_list_sub

def map_to_acronym(dict_):
    return_dict={}
    exception_count,exception_id=(0,[])
    for i in dict_:
        try:
            return_dict[id_to_acronym[i]]=dict_[i]
        except:
            exception_count+1
            exception_id.append(i)
            pass
    return return_dict

In [39]:
run=raw_input("Computationally intensive (roughly 6-8 hours) Run? (y/n)")
if run=="y":
    Master_injection_fraction_dictionary={}
    ec=0
    ecl=[]

    for i in experiment_list['id'].tolist():
        ecp,ec,ecl,ecs,ecls=generate_experiment_structure_fractions([i],annt,Substr,include_density=True)
        Master_injection_fraction_dictionary[i]=ecp[i]
        
elif run=="n":
    print "operation cancelled"


Computationally intensive (roughly 6-8 hours) Run? (y/n)y


2018-08-01 20:21:08,697 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/310035922?image=injection_density&resolution=25
2018-08-01 20:23:52,525 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/313397385?image=injection_density&resolution=25
2018-08-01 20:29:23,744 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/306561747?image=injection_density&resolution=25
2018-08-01 20:31:36,211 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/307135126?image=injection_density&resolution=25
2018-08-01 20:32:50,868 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/306804565?image=injection_density&resolution=25
2018-08-01 20:35:47,736 allensdk.api.api.retrieve_file_over_

2018-08-01 22:05:44,838 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/307163104?image=injection_density&resolution=25
2018-08-01 22:08:11,628 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/307137245?image=injection_density&resolution=25
2018-08-01 22:11:08,164 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/306958740?image=injection_density&resolution=25
2018-08-01 22:12:12,476 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/313387575?image=injection_density&resolution=25
2018-08-01 22:15:37,227 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/310552598?image=injection_density&resolution=25
2018-08-01 22:21:03,776 allensdk.api.api.retrieve_file_over_

### Save & Load station

In [40]:
"""SAVE"""
run=raw_input("Save? (y/n)")

if run=="y":
    with open ("Master_injection_fraction_dictionary",'wb') as f:
        pickle.dump(Master_injection_fraction_dictionary,f)
    
elif run=="n":
    print "operation cancelled"

Save? (y/n)y


In [32]:
"""Load"""
run=raw_input("Load? (y/n)")

if run=="y":
    Master_injection_fraction_dictionary={}
    with open("Master_injection_fraction_dictionary",'r') as f:
        Master_injection_fraction_dictionary=pickle.load(f)
elif run=="n":
    print "operation cancelled"

Load? (y/n)y


## Reality Check

make sure the fractions add up to no more than 1

#### Filter 


In [41]:
len(Master_injection_fraction_dictionary)

2995

In [42]:
exception_count=0
exception_list=[]

for i in experiment_list['id'].values.tolist():
    for j in Master_injection_fraction_dictionary[i]:
        if Master_injection_fraction_dictionary[i][j]>1:
            raise ValueError("A fraction cannot be larger than 1")


In [43]:
len(exception_list)

0

0 experiments are not valid keys for Master_injection_fraction_dictionary. Investigate

For good reasons, we wish to remove experiments that are too close to one another. First get a map from experiment id to their injection_coordinates

# Get Structure Volume

In [44]:
def volume_dictionary_gen(acr_list):
    """based on input acronym list, return a dictionary that maps ID to its volume.
    
        This is a poor workaround the fact that I don't know how to import the volume data of the structure from the website.
        I can only calculate the volume from the annotation files, but there are discrepencies between that and the unionization
        records.
    
    args:
        acr_list (list): input acronym list
        
    returns:
        returndict(dictionary): map from id to its volume
        exception_structure_acr (list) a list of structure that do not have the volume data available from this method"""
    
    returndict={}
    exception_struct_acr=[]
    
    for i in acr_list:
        
        '''volume data is acquired through unionization.
            From the experiment lists of a structure, I target itself and return its unionization records.
            '''
        struct=structure_tree.get_structures_by_acronym([i])[0]
        struct_experiments=mcc.get_experiments(cre=None,
                                              injection_structure_ids=[struct['id']])
        
        
        '''If error, then it must be the case that the structure does not belong to any injection site.
            I believe it is still possible for other structures to project to these regions, but for this particular analysis,
            I ignore these structures.'''
        try:
            structure_unionizes=mcc.get_structure_unionizes( [e['id'] for e in struct_experiments],
                                                          is_injection=None,
                                                          structure_ids=[struct['id']], # all
                                                          include_descendants=True,
                                                           hemisphere_ids=[2])
        except:
            exception_struct_acr.append(i)
            pass
        
        for j in range(len(structure_unionizes)):
            a=structure_unionizes['structure_id'][j]
            b=structure_unionizes['volume'][j]

            if a not in returndict:
                returndict[a]=b

    return returndict,exception_struct_acr

In [45]:
structure_unionizes=mcc.get_structure_unionizes( [e for e in list(experiment_list['id'])],
                                                          is_injection=None,
                                                          structure_ids=None, # all
                                                          include_descendants=True,
                                                           hemisphere_ids=[2])

C:\Users\Chris OBrien\AppData\Local\conda\conda\envs\p2\lib\site-packages\allensdk\api\cache.py:335: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  data = reader(path)


In [46]:
run=raw_input("computationally demanding. run? (y/n)")

if run=="y":

    id_to_volume={}
    str_id=structure_unionizes['structure_id']
    str_vol=structure_unionizes['volume']
    for i in range(len(str_id)):
        if i not in id_to_volume.keys():
            id_to_volume[str_id[i]]=str_vol[i]
elif run =="n"
    rpint "operation cancelled"

In [48]:
len(id_to_volume)

848

In [49]:
"""SAVE"""
run=raw_input("Save? (y/n)")

if run=="y":
    with open ("id_to_volume",'wb') as f:
        pickle.dump(id_to_volume,f)
    
elif run=="n":
    print "operation cancelled"

Save? (y/n)y


In [ ]:
"""Load"""
run=raw_input("Load? (y/n)")

if run=="y":
    with open("id_to_volume",'r') as f:
        id_to_volume=pickle.load(f)
elif run=="n":
    print "operation cancelled"

In [50]:
id_to_inj_coord=structure_tree

In [51]:
sinj=structure_tree.get_structures_by_acronym(['SCm'])[0]

In [52]:
sinj_experiments=mcc.get_experiments(cre=None,
                                         injection_structure_ids=[sinj['id']])

In [53]:
unionizes=mcc.get_structure_unionizes([ep['id'] for ep in sinj_experiments],
                                         is_injection=False,
                                         structure_ids=[acronym_to_id['AP']],
                                         include_descendants=False,
                                         hemisphere_ids=[2])

In [54]:
unionizes

,hemisphere_id,id,is_injection,max_voxel_density,max_voxel_x,max_voxel_y,max_voxel_z,normalized_projection_volume,projection_density,projection_energy,projection_intensity,projection_volume,experiment_id,structure_id,sum_pixel_intensity,sum_pixels,sum_projection_pixel_intensity,sum_projection_pixels,volume
0,2,636051208,False,0.000000,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,305025440,207,2.229142e+09,23752800.0,0.000000,0.000000,0.029097
1,2,631773396,False,0.010808,12830,5060,5920,2.933789e-07,0.000006,0.000554,94.313469,1.707713e-07,292533477,207,1.712106e+09,23752800.0,13147.783203,139.405151,0.029097
2,2,634324454,False,0.000000,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,307656580,207,4.448998e+09,23752800.0,0.000000,0.000000,0.029097
3,2,636038636,False,0.000000,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,305446876,207,1.769696e+08,23752800.0,0.000000,0.000000,0.029097
4,2,631855933,False,0.000000,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,294174290,207,1.927530e+08,23752800.0,0.000000,0.000000,0.029097
5,2,629972063,False,0.051576,12760,4930,5840,1.489958e-05,0.000157,0.014939,95.313133,4.560632e-06,175158132,207,2.521453e+09,23752800.0,354847.437500,3722.964844,0.029097
6,2,631919895,False,0.021249,12950,5130,5730,3.502646e-06,0.000011,0.000997,94.791611,3.061630e-07,278511717,207,2.268692e+09,23752800.0,23691.171875,249.928986,0.029097
7,2,630384216,False,0.035377,12400,4850,5820,8.376689e-05,0.000110,0.016814,152.640976,3.205130e-06,182613651,207,8.299907e+08,23752800.0,399374.875000,2616.432861,0.029097
8,2,633672424,False,0.000000,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,126646502,207,1.794206e+09,23752800.0,0.000000,0.000000,0.029097
9,2,633854016,False,0.047275,12800,4980,5920,1.328558e-05,0.000178,0.009861,55.241928,5.193812e-06,146078721,207,9.165370e+08,23752800.0,234217.312500,4239.846680,0.029097


In [55]:
import csv 

expid_to_inj_coord={}
expid_to_inj_volume={}

with open("projection_search_results.csv",'rb') as f:
    full_proj_search=csv.reader(f,delimiter=',')
    
    for row in full_proj_search:
        expid_to_inj_coord[row[0]]=row[14]
        expid_to_inj_volume[row[0]]=row[7]

In [56]:
eic=expid_to_inj_coord

In [57]:
expid_to_injcoord_int={}
for i in eic:
    if eic[i]!='injection-coordinates':
        expid_to_injcoord_int[int(i)]=[int(j) for j in eic[i][1:-1].split(',')]

In [58]:
expid_to_injcoordsquare={}
eic=expid_to_injcoord_int
for i in eic.keys():
    expid_to_injcoordsquare[i]=eic[i][0]**2+eic[i][1]**2+eic[i][2]**2

In [59]:
eics=expid_to_injcoordsquare

In [61]:
expid_to_inj_volume_int={}
for i in expid_to_inj_volume:
    if expid_to_inj_volume[i]!='injection-volume':
        expid_to_inj_volume_int[int(i)]=float(expid_to_inj_volume[i])

inj volume

In [182]:
def sum_proj(inj,target,hemisphere=[2],r1=0.1,r2=1.0):
    """
    args:
        inj (str): acronmy of the injection site
        target (str): acronym of the target structures
    
    returns:
    """
    
    #if not reciprocal:
    #    pass
    #elif reciprocal:
    #    save=inj
    #    inj=target
    #    target=save
        
    #1. get projection search results
    sinj=structure_tree.get_structures_by_acronym([inj])[0]
    starget=structure_tree.get_structures_by_acronym([target])[0]
    
    sinj_experiments=mcc.get_experiments(cre=None,
                                         injection_structure_ids=[sinj['id']])
    
    
    
    if len(sinj_experiments)==0:
        raise ValueError( 'No experiments exist for this structure')
        
    unionizes=mcc.get_structure_unionizes([ep['id'] for ep in sinj_experiments],
                                         is_injection=False,
                                         structure_ids=[starget['id']],
                                         include_descendants=False,
                                         hemisphere_ids=hemisphere)
    exception_count=0.0
    
    #2. using unionized records, get experiments id and its associated projection volume
    # from sinj to target
    a=unionizes['experiment_id']
    b=unionizes['projection_volume']
    
    #3. try fetching the volume data for the injection structure. return error otherwise
    try:
        sinj_structure_vol=float(id_to_volume[acronym_to_id[inj]])/2
    except:
        raise ValueError('Volume data does not exist for this structure')
    
    _error,__error,___error=0.0,0.0,0.0
    
    proj_sum=0.0
    tot_injection_volume_fraction=0.0
    
    for i in range(len(a)):
        p=i
        
        if float(expid_to_inj_volume_int[a[p]])>=r1 and float(expid_to_inj_volume_int[a[p]])<=r2:
            #proj sum = proj volume * fraction of the sinj region
            c1=float(b[p])
            #c2=float(Master_injection_fraction_dictionary[a[p]][acronym_to_id[inj]])
            proj_sum+=c1
            
            c3=float(expid_to_inj_volume_int[a[p]])
            #c4=float(Master_injection_fraction_dictionary[a[p]][acronym_to_id[inj]])
            tot_injection_volume_fraction+=c3
            
    if tot_injection_volume_fraction>=sinj_structure_vol:
        #print "tot inj volume larger"
        #print "tot injection fraction",tot_injection_volume_fraction
        #print "structure vol",sinj_structure_vol
        proj_sum=92000*proj_sum*tot_injection_volume_fraction/sinj_structure_vol
        ___error="NA"
        #___error=(proj_sum*sinj_structure_vol/tot_injection_volume_fraction)*np.sqrt((0.10)**2+(__error/tot_injection_volume_fraction)**2+(_error/proj_sum)**2)
        return proj_sum,___error
    elif tot_injection_volume_fraction<=sinj_structure_vol:
        #print "tot inj volume smaller"
        #print "tot inj volume",tot_injection_volume_fraction
        #print "structure vol",sinj_structure_vol
        proj_sum=92000*proj_sum*tot_injection_volume_fraction/sinj_structure_vol
        ___error="NA"
        #___error=(proj_sum*sinj_structure_vol/tot_injection_volume_fraction)*np.sqrt((0.10)**2+(__error/tot_injection_volume_fraction)**2+(_error/proj_sum)**2)
        return proj_sum,___error
        
        
            
""" 
            #proj sum = proj volume * fraction of the sinj region
            c1=float(b[p])
            c2=float(Master_injection_fraction_dictionary[a[p]][acronym_to_id[inj]])
            proj_sum+=c1*c2
            _error+=c1*c2*np.sqrt( (0.10)**2 + (0.02)**2)
            
            #tot inj_vol_fraction= injection volume of exp * fraction of the sinj region
            
            c3=float(expid_to_inj_volume_int[a[p]])
            c4=float(Master_injection_fraction_dictionary[a[p]][acronym_to_id[inj]])
            tot_injection_volume_fraction+=c3*c4
            __error+=c3*c4*np.sqrt((0.05)**2 + (0.02)**2)
        else:
            pass
    if tot_injection_volume_fraction>=sinj_structure_vol:
        try:
            print "tot inj volume larger"
            print tot_injection_volume_fraction
            print sinj_structure_vol
            proj_sum=92000*proj_sum*tot_injection_volume_fraction/sinj_structure_vol
            ___error=(proj_sum*sinj_structure_vol/tot_injection_volume_fraction)*np.sqrt((0.10)**2+(__error/tot_injection_volume_fraction)**2+(_error/proj_sum)**2)
            return proj_sum,___error
        except:
            print 'exception occured'
    elif tot_injection_volume_fraction<=sinj_structure_vol:
        try:
            print "tot inj volume smaller"
            print tot_injection_volume_fraction
            print sinj_structure_vol
            proj_sum=92000*proj_sum*sinj_structure_vol/tot_injection_volume_fraction
            ___error=(proj_sum*sinj_structure_vol/tot_injection_volume_fraction)*np.sqrt((0.10)**2+(__error/tot_injection_volume_fraction)**2+(_error/proj_sum)**2)
            return proj_sum,___error
        
        except ZeroDivisionError:
            return 0,0,0,0
"""
None

### NOTE: avoid using CTX since its volume is ridicously small


In [382]:
#create list of structures
Cortex_list=[id_to_acronym[k] for k in Substr[acronym_to_id['Isocortex']]]
BS_nuclei=['SCs','SCm','PB','NTS','PAG']
HY_nuclei=[id_to_acronym[k] for k in Substr[acronym_to_id['HY']]]
TH_nuclei=[id_to_acronym[k] for k in Substr[acronym_to_id['TH']]]

#### BS to CTX

In [433]:
r1=0.1
r2=1.0

In [409]:
BS_to_CTX={}

for i in BS_nuclei:
    BS_to_CTX[i]={}
    for j in Cortex_list:
        try:
            BS_to_CTX[i][j]=sum_proj(i,j,r1=r1,r2=r2)[0]
        except:
            print "exception",i,j

In [410]:
for i in BS_to_CTX:
    print i,round(sum([BS_to_CTX[i][j] for j in BS_to_CTX[i]]),2)

SCm 71614.62
PB 2288996.35
NTS 332.41
SCs 8981.4
PAG 302999.47


#### CTX to BS

In [434]:
CTX_to_BS={}

for i in Cortex_list:
    CTX_to_BS[i]={}
    for j in BS_nuclei:
        try:
            CTX_to_BS[i][j]=sum_proj(i,j,r1=r1,r2=r2)[0]
        except:
            print "exception",i,j

exception PERI SCs
exception PERI SCm
exception PERI PB
exception PERI NTS
exception PERI PAG


In [435]:
CTX_to_BS.pop('PERI',None)

{}

In [446]:
for i in BS_nuclei:
    print round(sum([CTX_to_BS[j][i] for j in CTX_to_BS.keys()]),2)

3885011.58
11517818.81
261102.33
43251.54
4578819.02


In [398]:
for i in CTX_to_BS:
    print i,round(sum([CTX_to_BS[i][j] for j in CTX_to_BS[i]]),2)

VISC 448146.48
SS 1927754.04
ACA 3959033.84
AUD 414137.81
ILA 869132.09
AI 77975.13
TEa 0.0
MO 2649989.21
GU 1539.87
ECT 137089.71
VIS 10282252.17
FRP 10576.86
PTLp 147662.07
ORB 978561.55
RSP 471877.36
PL 238919.97


#### BS to HY

In [437]:
BS_to_HY={}

for i in BS_nuclei:
    BS_to_HY[i]={}
    for j in HY_nuclei:
        try:
            BS_to_HY[i][j]=sum_proj(i,j,r1=r1,r2=r2)[0]
        except:
            print "exception",i,j

In [447]:
for b in BS_nuclei:
    print round(sum(BS_to_HY[b][i] for i in HY_nuclei),2)

2513.23
171857.9
1586207.62
470.78
1283115.73


#### HY to BS

In [439]:
HY_to_BS={}

for i in HY_nuclei:
    HY_to_BS[i]={}
    for j in BS_nuclei:
        try:
            HY_to_BS[i][j]=sum_proj(i,j,r1=r1,r2=r2)[0]
        except:
            print "exception",i,j

exception ME SCs
exception ME SCm
exception ME PB
exception ME NTS
exception ME PAG


In [440]:
HY_to_BS.pop('ME',None)

{}

In [448]:
for b in BS_nuclei:
    print round(sum([HY_to_BS[i][b] for i in HY_to_BS]),2)

557274.04
15722071.37
2454173.32
928044.52
41903554.57


#### BS to TH

In [442]:
BS_to_TH={}

for i in BS_nuclei:
    BS_to_TH[i]={}
    for j in TH_nuclei:
        try:
            BS_to_TH[i][j]=sum_proj(i,j,r1=r1,r2=r2)[0]
        except:
            print "exception",i,j

In [449]:
for b in BS_nuclei:
    print round(sum(BS_to_TH[b][i] for i in TH_nuclei),2)

64833.11
601083.29
1132894.97
1115.99
927354.75


#### TH to BS

In [444]:
TH_to_BS={}

for i in TH_nuclei:
    TH_to_BS[i]={}
    for j in BS_nuclei:
        try:
            TH_to_BS[i][j]=sum_proj(i,j,r1=0.1,r2=1.5)[0]
        except:
            print "exception",i,j

In [450]:
for b in BS_nuclei:
    print round(sum([TH_to_BS[i][b] for i in TH_to_BS]),2)

308725.58
1712721.05
70923.13
5445.85
2048788.41


## ILM

In [226]:
sum_proj('ILM','Isocortex',r1=r1,r2=r2)

(6164244.820564197, 'NA')

In [227]:
sum_proj('ILM','TH',r1=r1,r2=r2)

(2580796.7193952, 'NA')

In [228]:
sum_proj('ILM','HY',r1=r1,r2=r2)

(1427266.2691949375, 'NA')

In [229]:
sum_proj('Isocortex','ILM',r1=r1,r2=r2)

(3383596.3011150034, 'NA')

In [230]:
sum_proj('TH','ILM',r1=r1,r2=r2)

(1843627.9539560545, 'NA')

In [231]:
sum_proj('HY','ILM',r1=r1,r2=r2)

(4044288.531278573, 'NA')

In [106]:
_regions=['AUD','VIS','MO','SS','ACA','ORB','AI','RSP','PTLp','FRP','GU','VISC','PL','ILA','TEa','PERI','ECT']
BS_nuclei=['SCs', 'SCm','PB','NTS','PAG','AP']


for i in BS_nuclei:
    summe=0.0
    for j in _regions:
        summe+=sum_proj(j,i,r1=0.1,r2=1.5)[0]
    print i+' to '+j+" "+str(summe)

SCs to PTLp 61134.6139182


KeyboardInterrupt: 

In [304]:
_regions=['ADP', 'PVp', 'PVpo', 'VLPO', 'AVP', 'AVPV', 'SCH', 'SFO', 'SBPV', 'MEPO', 'MPO', 'OV', 'DMH', 'PD', 'PS', 'VMPO', 'PVa', 'PVH', 'PVi', 'ARH', 'ASO', 'SO', 'RCH', 'LHA', 'LPO', 'PST', 'PSTN', 'STN', 'TU', 'ZI', 'PeF', 'PVHd', 'AHN', 'MBO', 'MPN', 'VMH', 'PH', 'PMd', 'PMv']
BS_nuclei=['SCs', 'SCm','PB','NTS','PAG','AP']


for i in BS_nuclei:
    summe=0.0
    for j in _regions:
        try:
            summe+=sum_proj(j,i,r1=0.1,r2=1.5)[0]
        except:
            None
    print 'HY'+' to '+i+" "+str(summe)

tot inj volume smaller
tot inj volume 0.0
structure vol 0.02743295625
tot inj volume larger
tot injection fraction 0.402797109575
structure vol 0.03433129875
tot inj volume smaller
tot inj volume 0.0
structure vol 0.04993884
tot inj volume smaller
tot inj volume 0.0
structure vol 0.01812951
tot inj volume smaller
tot inj volume 0.0
structure vol 0.030153375
tot inj volume larger
tot injection fraction 3.1599357258
structure vol 0.1531179538
tot inj volume larger
tot injection fraction 2.62867507429
structure vol 0.1036520912
tot inj volume larger
tot injection fraction 0.397480505527
structure vol 0.0520242212
tot inj volume larger
tot injection fraction 0.242690293431
structure vol 0.078535485
tot inj volume smaller
tot inj volume 0.0
structure vol 0.01293949125
tot inj volume larger
tot injection fraction 6.9864844154
structure vol 0.5690300224
tot inj volume larger
tot injection fraction 0.366235996
structure vol 0.154343385
tot inj volume larger
tot injection fraction 0.345275952
s

tot inj volume larger
tot injection fraction 0.242690293431
structure vol 0.078535485
tot inj volume smaller
tot inj volume 0.0
structure vol 0.01293949125
tot inj volume larger
tot injection fraction 6.9864844154
structure vol 0.5690300224
tot inj volume larger
tot injection fraction 0.366235996
structure vol 0.154343385
tot inj volume larger
tot injection fraction 0.345275952
structure vol 0.0458932138
tot inj volume larger
tot injection fraction 0.1261697668
structure vol 0.053636625
tot inj volume larger
tot injection fraction 3.0530966844
structure vol 0.14421141
tot inj volume larger
tot injection fraction 4.3478578276
structure vol 0.4960538912
tot inj volume larger
tot injection fraction 0.199931722331
structure vol 0.03600820125
tot inj volume larger
tot injection fraction 7.25521112633
structure vol 0.1958574688
tot inj volume larger
tot injection fraction 5.02541391624
structure vol 0.27998262635
tot inj volume larger
tot injection fraction 3.30888414999
structure vol 0.1120

In [280]:
sum_proj('BS','HY',r1=0.1,r2=1.5)[0]

81511424.42761406

In [290]:
_regions=['ADP', 'PVp', 'PVpo', 'VLPO', 'AVP', 'AVPV', 'SCH', 'SFO', 'SBPV', 'MEPO', 'MPO', 'OV', 'DMH', 'PD', 'PS', 'VMPO', 'PVa', 'PVH', 'PVi', 'ARH', 'ASO', 'SO', 'RCH', 'LHA', 'LPO', 'PST', 'PSTN', 'STN', 'TU', 'ZI', 'PeF', 'PVHd', 'AHN', 'MBO', 'MPN', 'VMH', 'PH', 'PMd', 'PMv']
BS_nuclei=['SCs', 'SCm','PB','NTS','PAG','AP']


for i in BS_nuclei:
    summe=0.0
    for j in _regions:
        try:
            summe+=sum_proj(i,j,r1=0.1,r2=1.5)[0]
        except:
            None
    print i+' to '+'HY'+" "+str(summe)

exception occured
exception occured
exception occured
exception occured
exception occured
SCs to HY 2454.48250879
SCm to HY 171806.573731
PB to HY 1585387.81205
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
NTS to HY 460.525654608
PAG to HY 1282826.07808
AP to HY 0.0


In [291]:
sum_proj('ILM','CTX',r1=0.1,r2=1.5)[0]

7277884.273278983

In [298]:
sum_proj('CTX','ILM',r1=0.1,r2=1.5)[0]

tot inj volume larger
tot injection fraction 181.077545256
structure vol 0.0430031743938


3842745704.0153522

In [302]:
sum_proj('CTX','SCm',r1=0.1,r2=1.5)[0]

tot inj volume larger
tot injection fraction 181.077545256
structure vol 0.0430031743938


614.7468377566435

In [303]:
614*181/0.043

2584511.6279069767

In [299]:
id_to_volume[acronym_to_id['HY']]

8.3802130048

## Mcmodels
Here we use Mcmodels and their available data to explore the connectivity strength and the information flow


Structure sets used by joseph, et al on 'High resolution data_diven model' uses intermediate level summary structures. This most likely corresponds to the structure set number 687527945 'Structures representing the subdivisions of the Mouse Brain'

#### Set equivalence

In [280]:
lightsaber=pd.DataFrame(structure_tree.get_structures_by_set_id([687527945]))
lightsaber['acronym'].values

fine_collection=[ i[1] for i in Voxel_connectivity_dict.keys() ]
for 

In [345]:
len(list(set(fine_collection)))

291

In [329]:
len(list(lightsaber['acronym'].values))

293

In [338]:
l1=list(lightsaber['acronym'].values)
l2=fine_collection

In [346]:
for i in l1:
    if i not in l2:
        print 'wtf',i

wtf ENTmv
wtf fiber tracts


In [347]:
for i in l2:
    if i not in l1:
        print 'wtf',i

l1 contains l2 by 2 elements

In [264]:
from mcmodels.core import VoxelModelCache

#create cache instance
cache = VoxelModelCache()

cache has a method(?) called get_connection_srength tha fetches the connection strength data. From such data, we calculate the information flow

In [300]:
Voxel_connectivity=cache.get_connection_strength()
Voxel_connectivity_dict=dict(Voxel_connectivity)

## Exploring Damasio Network
Here we explore the consciousness network proposed by Damasio, divided into two sections, one for the network within the brainstem, and one for between nuclei of the brainstem and the cerebral cortex

Create a list that contains Damasio's Network

In [306]:
from collections import Iterable

def flatten(lis):
    """Iterate over an arbitrary dimensional iterable and returns a list containing
    all the elements in the list"""
    
    for item in lis:
        if isinstance(item,Iterable) and not isinstance(item,basestring):
            for x in flatten(item):
                yield x
        else:
            yield item

    

In [307]:
Damasio_Network=[['SCs','SCm'],
                ['PB'],
                ['NTS'],
                ['PAG'],
                ['AP'],
                ]
for i in Substr[acronym_to_id['HY']]:
    toappend=[str(id_to_acronym[j]) for j in Substr[i]]
    Damasio_Network.append(toappend)

In [308]:
Substr[acronym_to_id['HY']]

[141, 157, 290, 467, 10671]

#### From HY to BS

In [309]:
flat_Damasio=list(flatten(Damasio_Network))

In [310]:
flat_Damasio

['SCs', 'SCm', 'PB', 'NTS', 'PAG', 'AP', 'ADP', 'PVp', 'PVpo', 'VLPO', 'AVP', 'AVPV', 'SCH', 'SFO', 'SBPV', 'MEPO', 'MPO', 'OV', 'DMH', 'PD', 'PS', 'VMPO', 'PVa', 'PVH', 'PVi', 'ARH', 'ASO', 'SO', 'RCH', 'LHA', 'LPO', 'PST', 'PSTN', 'STN', 'TU', 'ZI', 'PeF', 'PVHd', 'AHN', 'MBO', 'MPN', 'VMH', 'PH', 'PMd', 'PMv']

In [312]:
exception_dict={}
Damasio_dict={}

for i in flat_Damasio[6:]:
    for j in flat_Damasio[:6]:
        try:
            proj=Voxel_connectivity_dict[('ipsi',i)][j]*92000
            Damasio_dict[(i,j)]=proj
        except:
            if i not in exception_dict:
                exception_dict[i]=0
            else:
                exception_dict[i]+=1

In [314]:
exception_dict

{'PeF': 5, 'MBO': 5, 'VMPO': 5}

In [315]:
HY_to_BS={}
for i in flat_Damasio[:6]:
    HY_to_BS[('HY',i)]=0

for i in flat_Damasio[6:]:
    for j in flat_Damasio[:6]:
        if i not in exception_dict.keys():
            add=Damasio_dict[(i,j)]
            HY_to_BS[('HY',j)]+=add

In [316]:
exception_dict.keys()

['PeF', 'MBO', 'VMPO']

In [318]:
%pprint

Pretty printing has been turned ON


In [319]:
HY_to_BS

{('HY', 'AP'): 601992.8089413116,
 ('HY', 'NTS'): 2888747.214828966,
 ('HY', 'PAG'): 112840524.61648126,
 ('HY', 'PB'): 35070641.28208359,
 ('HY', 'SCm'): 98010599.76085003,
 ('HY', 'SCs'): 15164716.876646407}

#### From BS to HY

In [321]:
exception_dict={}
Damasio_dict={}

for i in flat_Damasio[:6]:
    for j in flat_Damasio[6:]:
        try:
            proj=Voxel_connectivity_dict[('ipsi',i)][j]*92000
            Damasio_dict[(i,j)]=proj
        except:
            if i not in exception_dict:
                exception_dict[j]=0
            else:
                exception_dict[j]+=1

In [322]:
Damasio_dict

{('AP', 'ADP'): 312.9441143682925,
 ('AP', 'AHN'): 5135.985342581989,
 ('AP', 'ARH'): 3020.6431963597424,
 ('AP', 'ASO'): 17.178109206724912,
 ('AP', 'AVP'): 173.4274537739111,
 ('AP', 'AVPV'): 355.974893682287,
 ('AP', 'DMH'): 3795.1260693953373,
 ('AP', 'LHA'): 11433.845429819485,
 ('AP', 'LPO'): 1416.3174082677872,
 ('AP', 'MEPO'): 205.06868507163747,
 ('AP', 'MPN'): 2100.520661333576,
 ('AP', 'MPO'): 1860.2410752355354,
 ('AP', 'OV'): 22.933276151889004,
 ('AP', 'PD'): 57.28061546687968,
 ('AP', 'PH'): 2095.3289983444847,
 ('AP', 'PMd'): 587.8298965108115,
 ('AP', 'PMv'): 1563.8487985997926,
 ('AP', 'PS'): 310.3679866471793,
 ('AP', 'PST'): 45.41565314866603,
 ('AP', 'PSTN'): 336.65115421536024,
 ('AP', 'PVH'): 1833.1710154307077,
 ('AP', 'PVHd'): 1210.4691684362479,
 ('AP', 'PVa'): 19.303322042105716,
 ('AP', 'PVi'): 588.5014634695835,
 ('AP', 'PVp'): 790.4466252075508,
 ('AP', 'PVpo'): 546.6821859154152,
 ('AP', 'RCH'): 876.9786358170678,
 ('AP', 'SBPV'): 1063.4426152973904,
 ('A

In [323]:
exception_dict

{'MBO': 0, 'PeF': 0, 'VMPO': 0}

In [324]:
BS_to_HY={}
for i in flat_Damasio[:6]:
    BS_to_HY[('BS',i)]=0

for i in flat_Damasio[:6]:
    for j in flat_Damasio[6:]:
        if j not in exception_dict.keys():
            add=Damasio_dict[(i,j)]
            BS_to_HY[('BS',i)]+=add

In [325]:
BS_to_HY

{('BS', 'AP'): 57174.72434903448,
 ('BS', 'NTS'): 1940701.527778525,
 ('BS', 'PAG'): 260257660.69746017,
 ('BS', 'PB'): 10665332.4003499,
 ('BS', 'SCm'): 173464864.34695125,
 ('BS', 'SCs'): 5235818.558663945}

In [215]:
HY_to_BS

{('HY', 'AP'): 601992.8089413116,
 ('HY', 'NTS'): 2888747.214828966,
 ('HY', 'PAG'): 112840524.61648126,
 ('HY', 'PB'): 35070641.28208359,
 ('HY', 'SCm'): 98010599.76085003,
 ('HY', 'SCs'): 15164716.876646407}

## Within BS

In [221]:
WithinBS=flat_Damasio[:6]
BS_dict={}

In [224]:
for i in WithinBS:
    for j in WithinBS:
        proj=Voxel_connectivity_dict[('ipsi',i)][j]*92000
        BS_dict[(i,j)]=proj
        

## Between CTX and HypoBS

We want to be able to create a table of information flow in terms of 'large' cortical regions, such as VIS, AUD etc. So we need a way to manage that. 

The previously defined Substr and Unpack may come in handy here



In [337]:
Voxel_structures.shape

(291, 8)

In [338]:
def Cortical_HYBS_map(ctx_region,target):
    """Calculates the Projection sum from the specific cortical regions into the BS or the HyThalamus
    args:
        ctx_region(basestring): id of the structure
        target(basestring): id of the target structure"""
    
    source=[id_to_acronym[i] for i in Unpack(acronym_to_id[ctx_region],Substr,include_self=False)] 
            #list of id
    
    #if it has no substructure
    #if len(source)==0:
    #    source=Unpack(ctx_region,Substr,include_self=True)
    
    Voxel_source=[]
    for i in source:
        if i in list(Voxel_structures['acronym']):
            Voxel_source.append(i)
    
    ret=0
    for i in Voxel_source:
        ret+=Voxel_connectivity_dict[('ipsi',i)][target]*92000
    return ret
    

In [340]:
Unpack(acronym_to_id['Isocortex'],Substr)

[22,
 312782546,
 312782550,
 312782554,
 312782558,
 312782562,
 312782566,
 312782570,
 417,
 312782604,
 312782608,
 312782612,
 312782616,
 312782620,
 312782624,
 31,
 39,
 211,
 919,
 927,
 935,
 1015,
 48,
 296,
 588,
 772,
 810,
 819,
 44,
 556,
 707,
 827,
 1054,
 1081,
 95,
 104,
 328,
 783,
 831,
 996,
 1101,
 111,
 120,
 163,
 314,
 344,
 355,
 119,
 675,
 694,
 699,
 704,
 800,
 184,
 68,
 667,
 526157192,
 526157196,
 526322264,
 247,
 1002,
 251,
 735,
 816,
 847,
 954,
 1005,
 1011,
 156,
 243,
 252,
 527,
 600,
 678,
 1018,
 520,
 598,
 755,
 959,
 990,
 1023,
 1027,
 249,
 456,
 643,
 696,
 759,
 791,
 254,
 879,
 274,
 330,
 434,
 442,
 545,
 610,
 886,
 430,
 542,
 590,
 622,
 687,
 894,
 279,
 671,
 774,
 906,
 965,
 500,
 985,
 320,
 648,
 844,
 882,
 943,
 993,
 656,
 767,
 962,
 1021,
 1085,
 541,
 97,
 234,
 289,
 729,
 786,
 1127,
 669,
 385,
 33,
 305,
 593,
 721,
 778,
 821,
 394,
 281,
 401,
 433,
 441,
 1046,
 1066,
 402,
 233,
 601,
 649,
 905,
 1074,
 11

In [341]:
CTX_regions=['AUD','VIS','MO','SS','ACA','ORB','AI','RSP','PTLp']
BS_nuclei=['SCs', 'SCm','PB','NTS','PAG','AP']

CTX_to_BS_nuclei={}
for i in BS_nuclei:
    CTX_to_BS_nuclei[i]=0
    
sum_me=0
for i in BS_nuclei:
    for j in CTX_regions:
        addme=Cortical_HYBS_map(j,i)
        sum_me+=addme
    CTX_to_BS_nuclei[i]=sum_me
    sum_me=0

In [342]:

CTX_to_BS_nuclei

{'AP': 565280.6168194802,
 'NTS': 3127679.3021562,
 'PAG': 7156091.618768791,
 'PB': 2035798.1088416118,
 'SCm': 23363330.75415419,
 'SCs': 15872781.214532183}

In [343]:
CTX_regions=['AUD','VIS','MO','SS','ACA','ORB','AI','RSP','PTLp']
HY_nuclei=flat_Damasio[6:]

exception_list=['VMPO','PeF','MBO']


CTX_to_HY_nuclei={}
for i in HY_nuclei:
    CTX_to_HY_nuclei[i]=0
    
sum_me=0
for i in HY_nuclei:
    for j in CTX_regions:
        if i not in exception_list:
            addme=Cortical_HYBS_map(j,i)
            sum_me+=addme
        else:
            None
    CTX_to_HY_nuclei[i]=sum_me
    sum_me=0

CTX into HYpothalamus

In [344]:
sum(CTX_to_HY_nuclei.values())

129265943.04776353

In [345]:
CTX_to_HY_nuclei

{'ADP': 538049.9786980799,
 'AHN': 3632501.6386664286,
 'ARH': 550850.4953427764,
 'ASO': 7595.66878847545,
 'AVP': 750974.4164174772,
 'AVPV': 1304203.5219089012,
 'DMH': 1286524.5023722383,
 'LHA': 19129745.851558343,
 'LPO': 5313415.601819201,
 'MBO': 0,
 'MEPO': 370395.5531149749,
 'MPN': 2838825.0146697685,
 'MPO': 4641934.4503163155,
 'OV': 70125.4159459204,
 'PD': 67555.88975602586,
 'PH': 4878384.240038606,
 'PMd': 750161.4825464785,
 'PMv': 722299.6134909481,
 'PS': 842841.3298039231,
 'PST': 339992.0882400547,
 'PSTN': 3475675.774695992,
 'PVH': 711798.9605593029,
 'PVHd': 473235.91913527343,
 'PVa': 18123.759165217052,
 'PVi': 185419.64702977566,
 'PVp': 525430.5061530467,
 'PVpo': 686083.7776765111,
 'PeF': 0,
 'RCH': 560383.9958829485,
 'SBPV': 648672.9450600542,
 'SCH': 349952.2626435937,
 'SFO': 0.0,
 'SO': 360397.34653761843,
 'STN': 8656137.997652888,
 'TU': 930343.8001445829,
 'VLPO': 772891.6470502154,
 'VMH': 1030423.5835245054,
 'VMPO': 0,
 'ZI': 61844594.371357076

In [346]:
sum(CTX_to_HY_nuclei.values())

129265943.04776353

In [331]:
Voxel_structures=pd.DataFrame(structure_tree.get_structures_by_set_id([687527945]))
Voxel_structures[Voxel_structures=='fiber tracts'].index

RangeIndex(start=0, stop=293, step=1)

Recall that we need to remove fiber tracts and ENTmv. So cleam em up

#### BS to TH

In [359]:
Substr[acronym_to_id['TH']]

[856, 864]

In [361]:
Unpack(acronym_to_id['TH'],Substr)

[856, 51, 189, 575, 599, 907, 930, 560581563, 138, 218, 325, 1020, 1029, 560581551, 239, 64, 127, 1096, 1104, 155, 255, 1113, 1120, 262, 444, 59, 362, 366, 1077, 571, 15, 149, 181, 560581559, 958, 186, 483, 1014, 27, 178, 321, 563807439, 864, 406, 414, 422, 609, 637, 685, 563807435, 629, 709, 718, 725, 733, 741, 1008, 170, 496345664, 496345668, 496345672, 475, 1072, 1079, 1088, 1044]

In [365]:
Damasio_Network=[['SCs','SCm'],
                ['PB'],
                ['NTS'],
                ['PAG'],
                ['AP'],
                ]
Damasio_Network=Damasio_Network+[id_to_acronym[i] for i in Unpack(acronym_to_id['TH'],Substr)]
#for i in Substr[acronym_to_id['TH']]:
#    toappend=[str(id_to_acronym[j]) for j in Substr[i]]
#    Damasio_Network.append(toappend)

In [366]:
Damasio_Network

[['SCs', 'SCm'], ['PB'], ['NTS'], ['PAG'], ['AP'], u'DORpm', u'ILM', u'RH', u'CL', u'CM', u'PCN', u'PF', u'PIL', u'LAT', u'LP', u'SGN', u'PO', u'POL', u'Eth', u'ATN', u'AD', u'AM', u'AMd', u'AMv', u'LD', u'AV', u'IAD', u'IAM', u'RT', u'MED', u'IMD', u'MD', u'SMT', u'PR', u'MTN', u'PT', u'PVT', u'RE', u'Xi', u'EPI', u'LH', u'MH', u'GENv', u'IGL', u'LGv', u'SubG', u'IntG', u'DORsm', u'SPF', u'SPFm', u'SPFp', u'SPA', u'VENT', u'VM', u'PoT', u'VAL', u'VP', u'VPL', u'VPLpc', u'VPM', u'VPMpc', u'GENd', u'LGd', u'LGd-sh', u'LGd-co', u'LGd-ip', u'MG', u'MGd', u'MGv', u'MGm', u'PP']

In [368]:
flat_Damasio=list(flatten(Damasio_Network))
flat_Damasio

['SCs', 'SCm', 'PB', 'NTS', 'PAG', 'AP', u'DORpm', u'ILM', u'RH', u'CL', u'CM', u'PCN', u'PF', u'PIL', u'LAT', u'LP', u'SGN', u'PO', u'POL', u'Eth', u'ATN', u'AD', u'AM', u'AMd', u'AMv', u'LD', u'AV', u'IAD', u'IAM', u'RT', u'MED', u'IMD', u'MD', u'SMT', u'PR', u'MTN', u'PT', u'PVT', u'RE', u'Xi', u'EPI', u'LH', u'MH', u'GENv', u'IGL', u'LGv', u'SubG', u'IntG', u'DORsm', u'SPF', u'SPFm', u'SPFp', u'SPA', u'VENT', u'VM', u'PoT', u'VAL', u'VP', u'VPL', u'VPLpc', u'VPM', u'VPMpc', u'GENd', u'LGd', u'LGd-sh', u'LGd-co', u'LGd-ip', u'MG', u'MGd', u'MGv', u'MGm', u'PP']

In [369]:
exception_dict={}
Damasio_dict={}

for i in flat_Damasio[:6]:
    for j in flat_Damasio[6:]:
        try:
            proj=Voxel_connectivity_dict[('ipsi',i)][j]*92000
            Damasio_dict[(i,j)]=proj
        except:
            if i not in exception_dict:
                exception_dict[j]=0
            else:
                exception_dict[j]+=1

In [370]:
exception_dict

{u'VENT': 0, u'ATN': 0, u'DORpm': 0, u'AMd': 0, u'EPI': 0, u'GENd': 0, u'AMv': 0, u'PIL': 0, u'SPF': 0, u'MGm': 0, u'IntG': 0, u'ILM': 0, u'MGd': 0, u'GENv': 0, u'PoT': 0, u'DORsm': 0, u'VP': 0, u'LGd-ip': 0, u'MTN': 0, u'MGv': 0, u'Eth': 0, u'LGd-co': 0, u'MED': 0, u'Xi': 0, u'LGd-sh': 0, u'LAT': 0}

In [372]:
BS_to_TH={}
for i in flat_Damasio[:6]:
    BS_to_TH[('BS',i)]=0

for i in flat_Damasio[:6]:
    for j in flat_Damasio[6:]:
        if j not in exception_dict.keys():
            add=Damasio_dict[(i,j)]
            BS_to_TH[('BS',i)]+=add

In [374]:
%pprint
BS_to_TH

Pretty printing has been turned ON


{('BS', 'AP'): 12460.166371123903,
 ('BS', 'NTS'): 29654.919629143642,
 ('BS', 'PAG'): 34568154.08296166,
 ('BS', 'PB'): 1141972.0820060587,
 ('BS', 'SCm'): 23268523.906333372,
 ('BS', 'SCs'): 6760251.82519597}

#### TH to BS

In [375]:
exception_dict={}
Damasio_dict={}

for i in flat_Damasio[6:]:
    for j in flat_Damasio[:6]:
        try:
            proj=Voxel_connectivity_dict[('ipsi',i)][j]*92000
            Damasio_dict[(i,j)]=proj
        except:
            if i not in exception_dict:
                exception_dict[i]=0
            else:
                exception_dict[i]+=1

In [377]:
TH_to_BS={}
for i in flat_Damasio[:6]:
    TH_to_BS[('TH',i)]=0

for i in flat_Damasio[6:]:
    for j in flat_Damasio[:6]:
        if i not in exception_dict.keys():
            add=Damasio_dict[(i,j)]
            TH_to_BS[('TH',j)]+=add

In [378]:
TH_to_BS

{('TH', 'AP'): 679336.9869890029,
 ('TH', 'NTS'): 3525164.047008895,
 ('TH', 'PAG'): 138277715.4782364,
 ('TH', 'PB'): 22941761.250838745,
 ('TH', 'SCm'): 328942051.553909,
 ('TH', 'SCs'): 144937342.76368484}

#### Clean em up

In [334]:
indexfinder=Voxel_structures['acronym']
indexfinder[indexfinder=='fiber tracts']

247    fiber tracts
Name: acronym, dtype: object

In [335]:
indexfinder[indexfinder=='ENTmv']

226    ENTmv
Name: acronym, dtype: object

In [336]:
Voxel_structures=Voxel_structures.drop([247,226]) # drop em bois

In [393]:
Voxel_structures.shape

(291, 8)

nice

In [117]:
"""Substr_Voxel={} # key: structure id, value =[] 
for i in Voxel_structures['acronym']:
    if i not in Substr_Voxel.keys():
        Substr_Voxel[acronym_to_id[i]]=[]


idpath=Voxel_structures['structure_id_path']
for i in idpath:
    print i[-2]
    try:
        Substr_Voxel[i[-2]].append(i[-1])
    except:
        print i"""

"Substr_Voxel={} # key: structure id, value =[] \nfor i in Voxel_structures['acronym']:\n    if i not in Substr_Voxel.keys():\n        Substr_Voxel[acronym_to_id[i]]=[]\n\n\nidpath=Voxel_structures['structure_id_path']\nfor i in idpath:\n    print i[-2]\n    try:\n        Substr_Voxel[i[-2]].append(i[-1])\n    except:\n        print i"

In [409]:
id_to_name[557]

u'Tuberomammillary nucleus'

## Completing Damasio Network

In [ ]:
[
('CTX','HY'),
('CTX','PB'),
('CTX','PAG'),
('CTX','SCm'),
('CTX','SCm')
('HY','PAG'),
('HY','NTS'),
('PAG','SCm'),
('PAG','SCs'),
('PAG','PB'),
('PAG','NTS'),
('PB','AP'),
('PB','NTS')

]

In [398]:
for i in CTX_to_BS:
    print i,round(sum([CTX_to_BS[i][j] for j in CTX_to_BS[i]]),2)

VISC 448146.48
SS 1927754.04
ACA 3959033.84
AUD 414137.81
ILA 869132.09
AI 77975.13
TEa 0.0
MO 2649989.21
GU 1539.87
ECT 137089.71
VIS 10282252.17
FRP 10576.86
PTLp 147662.07
ORB 978561.55
RSP 471877.36
PL 238919.97


In [463]:
Voxel_connectivity_dict[('ipsi','SCm')]['PB']*92000

8485686.30076945

In [458]:
Voxel_connectivity_dict.keys()[550]

('ipsi', 'VCO')

# Homogeneous model

In [1]:
from mcmodels.core import VoxelModelCache, RegionalData
from mcmodels.models import HomogeneousModel

In [ ]:
# get data with which to fit model

cache = VoxelModelCache()
voxel_data = cache.get_experiment_data()

In [ ]:
regional_data = RegionalData.from_voxel_data(voxel_data)

In [92]:
reg = HomogeneousModel()
reg.fit(regional_data.injections, regional_data.projections)
HomogeneousModel(kappa=1000)

HomogeneousModel(kappa=1000)

In [94]:
HomogeneousModel.weights

# mcmodel: mesoscale mouse model data